In [4]:
!source activate py36

In [5]:
import sys

sys.path.append('/home/faske/g/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/home/faske/g/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
#import Levenshtein as lv

In [6]:
root = "/home/faske/g/milkweed"

In [7]:
cd $root

/working/tfaske/milkweed


In [8]:
pwd

'/working/tfaske/milkweed'

# Actual Mapping 
 

In [9]:
fastq_files = []
for seqdir in ['demult/fastq/']:
    os.chdir('{}/{}'.format(root, seqdir))
    files = !find . -name 'FA*fastq.gz'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        fastq_files.append(x)
fastq_files = sorted(fastq_files)

In [10]:
len(fastq_files)

176

In [11]:
cd $root

/working/tfaske/milkweed


In [12]:
assembly = "/home/faske/g/milkweed/assembly/Asclepias_syriaca_v0.3_chromosomes_and_contigs.fasta"

#indexed as: bowtie2-build --threads 10 -f Asclepias_syriaca_v0.3_chromosomes_and_contigs.fasta Asclepias_syriaca_v0.3_chromosomes_and_contigs.fasta

In [13]:
# --very-fast-local
# Same as: -D 5 -R 1 -N 0 -L 25 -i S,1,2.00

# --fast-local
# Same as: -D 10 -R 2 -N 0 -L 22 -i S,1,1.75

# --sensitive-local
# Same as: -D 15 -R 2 -N 0 -L 20 -i S,1,0.75 (default in --local mode)

# --very-sensitive-local
# Same as: -D 20 -R 3 -N 0 -L 20 -i S,1,0.50

#@lview.remote()
def run_bowtie2(args):
    import os, multiprocessing, socket #stopwatch,
    #timer = stopwatch.Timer()
    cpus = 12
    assembly, reads, outdir = args
    sam = os.path.join(outdir, "{}.sam".format(os.path.basename(reads)))
    #sam = tempfile.NamedTemporaryFile(delete=False, dir="/tmp")
    #print (sam)
    cmd = "bowtie2 --local -D 20 -R 3 -N 1 -L 20 -i S,1,0.50 -p %d -x %s -U %s -S %s" % (cpus,
                                                               assembly,
                                                               reads,
                                                               sam)
    res = None
    res = cmd
#     if not os.path.exists(sam):
#         res = !$cmd
    #timer.stop()
    return  cmd #,timer.elapsed

In [14]:
bowtie_dir = "/home/faske/g/milkweed/SNPcall/bowtie2/"
assert(bowtie_dir)

In [15]:
res = []
for f in fastq_files:
    r = run_bowtie2((assembly, f, bowtie_dir))
    res.append(r)

In [16]:
res[0]

'bowtie2 --local -D 20 -R 3 -N 1 -L 20 -i S,1,0.50 -p 12 -x /home/faske/g/milkweed/assembly/Asclepias_syriaca_v0.3_chromosomes_and_contigs.fasta -U /working/tfaske/milkweed/demult/fastq/FA_BM_607.fastq.gz -S /home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_607.fastq.gz.sam'

In [17]:
pwd

'/working/tfaske/milkweed'

In [18]:
def write_bowtie_sh(cmds):
    with open("run_bowtie.sh", "w") as o:
        o.write("""#!/bin/bash
#$ -S /bin/bash
#$ -cwd
#$ -o bowtie2.out
#$ -e bowtie2.err
source activate py36\n""")
        for elem in cmds:
            o.write("{}\n".format(elem))

In [19]:
cd $root

/working/tfaske/milkweed


In [20]:
write_bowtie_sh(res)

# Run run_bowie.sh
    mv run_bowtie.sh SNPcall 
    cd SNPcall
    bash run_bowtie.sh > bowtie.out &

In [32]:
fqs = !find /working/tfaske/milkweed/demult/fastq/ -name "FA*fastq.gz"

In [33]:
sams = !find $bowtie_dir -name "*.sam"

In [34]:
len(fqs), len(sams)

(176, 176)

In [35]:
sams

['/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_607.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_608.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_609.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_610.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_611.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_612.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_613.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_614.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_615.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_616.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_635.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_636.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_637.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_638.fastq.gz.sam',
 '/home/faske/g/milkweed/SNPcall/bowtie2/FA_BM_639.fastq.gz.sam',
 '/home/fa

In [36]:
SNPcall_dir = "/home/faske/g/milkweed/SNPcall/"
assert(SNPcall_dir)

In [37]:
cd $SNPcall_dir

/working/tfaske/milkweed/SNPcall


In [38]:
def write_sam2bam_sh(sam):
    with open("./run_sam2bam.sh", "w") as o:
        o.write("""#!/bin/bash
#$ -S /bin/bash
#$ -cwd
#$ -o sam2bam.out
#$ -e sam2bam.err
source activate py36\n""")
        for s in sam:
            bam = s.replace(".sam", ".bam")
            bam_sorted = "%s_sorted.bam" % bam.replace(".bam", "")
            o.write("samtools view -b {0} -o {1}\nsamtools sort -@ 8 {1} -o {2}\nsamtools index {2}\n".format(s,bam,bam_sorted))

In [39]:
write_sam2bam_sh(sams)

# run_sam2bam.sh
    cd /working/tfaske/milkweed/SNPcall 
    bash run_sam2bam.sh &> sam2bam.out & 
    disown -h 
    close window


In [40]:
cd $bowtie_dir

/working/tfaske/milkweed/SNPcall/bowtie2


In [44]:
sorted_bams = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    sorted_bams.append(x)
sorted_bams = sorted(sorted_bams)

#assert len(sorted_bams) == len(fastq_files)

len(sorted_bams), len(fqs), len(sams)
#missing 3 bam files from lane1

(158, 176, 176)